Based on [Variational Quantum Eigensolver (VQE) | PennyLane Tutorial](https://www.youtube.com/watch?v=4Xnxa6tzPeA)
[Molecure geometry optimization](https://pennylane.ai/qml/demos/tutorial_mol_geo_opt/)
[A brief overview of VQE](https://pennylane.ai/qml/demos/tutorial_vqe/)
Useful resources:

[Variantional Qunatum EigenSolver](https://github.com/qiskit-community/qgss-2023/blob/main/Lecture%20Notes/Lecture%208.2%20-%20Variational%20Quantum%20Eigensolver.pdf)
![VQE](./images/VQE_2.png)

VQE goal
------------------
The goal of the variational algorithm is to find the global minimum of
the cost function $g(\theta, x) = \langle \Psi(\theta) \vert H(x) \vert \Psi(\theta) \rangle$ with respect to the circuit parameters $\theta$ and the nuclear
coordinates $x$ entering the electronic Hamiltonian of the molecule.

In this tutorial we demonstrate how to use PennyLane to implement
quantum optimization of molecular geometries. The algorithm consists of
the following steps:

1.  Build the parametrized electronic Hamiltonian $H(x)$ of the
    molecule.

2.  Design the variational quantum circuit to prepare the electronic
    trial state of the molecule, $\vert \Psi(\theta) \rangle$.

3.  Define the cost function
    $g(\theta, x) = \langle \Psi(\theta) \vert H(x) \vert
    \Psi(\theta) \rangle$.

4.  Initialize the variational parameters $\theta$ and $x$. Perform a
    joint optimization of the circuit and Hamiltonian parameters to
    minimize the cost function $g(\theta, x)$. The gradient with respect
    to the circuit parameters can be obtained using a variety of
    established methods, which are natively supported in PennyLane. The
    gradients with respect to the nuclear coordinates can be computed
    using the formula

    $$\nabla_x g(\theta, x) = \langle \Psi(\theta) \vert \nabla_x H(x) \vert \Psi(\theta) \rangle.$$

Once the optimization is finalized, the circuit parameters determine the
energy of the electronic state, and the nuclear coordinates determine
the equilibrium geometry of the molecule in this state.

Let\'s get started! ⚛️

In [22]:
from pennylane import numpy as np
import pennylane as qml

symbols = ["H", "H", "H"]
x = np.array([0.028, 0.054, 0.0, 0.986, 1.610, 0.0, 1.855, 0.002, 0.0], requires_grad=True)
def H(x):
    return qml.qchem.molecular_hamiltonian(symbols, x, charge=1)[0]

In [23]:
hf = qml.qchem.hf_state(electrons=2, orbitals=6)
print(hf)

[1 1 0 0 0 0]


In [24]:
num_wires = 6
dev = qml.device("lightning.qubit", wires=num_wires)


@qml.qnode(dev, interface="autograd")
def circuit(params, obs, wires):
    qml.BasisState(hf, wires=wires)
    qml.DoubleExcitation(params[0], wires=[0, 1, 2, 3])
    qml.DoubleExcitation(params[1], wires=[0, 1, 4, 5])

    return qml.expval(obs)

This circuit prepares the trial state
$$\vert\Psi(\theta_1, \theta_2)\rangle =
\mathrm{cos}(\theta_1)\mathrm{cos}(\theta_2)\vert110000\rangle -
\mathrm{cos}(\theta_1)\mathrm{sin}(\theta_2)\vert000011\rangle -
\mathrm{sin}(\theta_1)\vert001100\rangle,$$

The cost function and the nuclear gradients
===========================================

The third step of the algorithm is to define the cost function
$g(\theta, x) = \langle \Psi(\theta) \vert H(x) \vert\Psi(\theta) \rangle$.
It evaluates the expectation value of the parametrized Hamiltonian
$H(x)$ in the trial state $\vert\Psi(\theta)\rangle$.

Next, we define the `cost` function $g(\theta, x)$ which depends on both the circuit and the Hamiltonian parameters. Specifically we consider the expectation values of the Hamiltonian.

In [25]:
def cost(params,x):
    hamiltonian = H(x)
    return circuit(params,obs=hamiltonian,wires=range(num_wires))

We use the `finite_diff()` function tocompute the gradient of the Hamiltonian using a central-difference approximation. Then, we evaluate the expectation value of the gradient components $\frac{\partial H(x)}{\partial x_i}$. This is implemented by the function `grad_x`:

In [26]:
def finite_diff(f, x, delta=0.01):
    """Compute the central-difference finite difference of a function"""
    gradient = []

    for i in range(len(x)):
        shift = np.zeros_like(x)
        shift[i] += 0.5 * delta
        res = (f(x + shift) - f(x - shift)) * delta**-1
        gradient.append(res)

    return gradient


def grad_x(params, x):
    grad_h = finite_diff(H, x)
    grad = [circuit(params, obs=obs, wires=range(num_wires)) for obs in grad_h]
    return np.array(grad)

Optimization of the molecular geometry
======================================

Finally, we proceed to minimize our cost function to find the ground
state equilibrium geometry of the $\mathrm{H}_3^+$ molecule. As a
reminder, the circuit parameters and the nuclear coordinates will be
jointly optimized at each optimization step. This approach does not
require nested VQE optimization of the circuit parameters for each set
of nuclear coordinates.

We start by defining the classical optimizers:

In [28]:
opt_theta = qml.GradientDescentOptimizer(stepsize=0.4)
opt_x = qml.GradientDescentOptimizer(stepsize=0.8)

In [29]:
theta = np.array([0.0, 0.0], requires_grad=True)

In [30]:
from functools import partial

# store the values of the cost function
energy = []

# store the values of the bond length
bond_length = []

# Factor to convert from Bohrs to Angstroms
bohr_angs = 0.529177210903

for n in range(100):

    # Optimize the circuit parameters
    theta.requires_grad = True
    x.requires_grad = False
    theta, _ = opt_theta.step(cost, theta, x)

    # Optimize the nuclear coordinates
    x.requires_grad = True
    theta.requires_grad = False
    _, x = opt_x.step(cost, theta, x, grad_fn=grad_x)

    energy.append(cost(theta, x))
    bond_length.append(np.linalg.norm(x[0:3] - x[3:6]) * bohr_angs)

    if n % 4 == 0:
        print(f"Step = {n},  E = {energy[-1]:.8f} Ha,  bond length = {bond_length[-1]:.5f} A")

    # Check maximum component of the nuclear gradient
    if np.max(grad_x(theta, x)) <= 1e-05:
        break

print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")
print("\n" "Ground-state equilibrium geometry")
print("%s %4s %8s %8s" % ("symbol", "x", "y", "z"))
for i, atom in enumerate(symbols):
    print(f"  {atom}    {x[3 * i]:.4f}   {x[3 * i + 1]:.4f}   {x[3 * i + 2]:.4f}")

Step = 0,  E = -1.26094338 Ha,  bond length = 0.96762 A
Step = 4,  E = -1.27360653 Ha,  bond length = 0.97619 A
Step = 8,  E = -1.27437809 Ha,  bond length = 0.98223 A
Step = 12,  E = -1.27443305 Ha,  bond length = 0.98457 A
Step = 16,  E = -1.27443729 Ha,  bond length = 0.98533 A
Step = 20,  E = -1.27443763 Ha,  bond length = 0.98556 A
Step = 24,  E = -1.27443766 Ha,  bond length = 0.98563 A

Final value of the ground-state energy = -1.27443766 Ha

Ground-state equilibrium geometry
symbol    x        y        z
  H    0.0102   0.0442   0.0000
  H    0.9867   1.6303   0.0000
  H    1.8720   -0.0085   0.0000



![chart](./images/VQE_1.png)

![chart](./images/VQE_5.png)

We're going to use VQE to find ground state of this three molecular.
We'll do this by three steps:
1. Find molecular Hamiltonian
2. Prepare trial ground state
3. Minimize $\langle \hat{H} \rangle$

# 1. Finding molecular Hamiltonian


In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem

In [2]:
symbols = ["H","H","H"]
coordinates = np.array([[0.028, 0.054, 0.0], [0.986, 1.610, 0.0],[ 1.855, 0.002, 0.0]]) # where are located
hamiltonian, qubits = qchem.molecular_hamiltonian(symbols,coordinates,charge=1)

In [3]:
print(qubits)

6


In [4]:
hf = qchem.hf_state(electrons=2,orbitals=6)#Hartree -Fock State
print(hf)

[1 1 0 0 0 0]


In [5]:
num_wires = qubits
dev = qml.device("default.qubit",wires = num_wires)

In [6]:
@qml.qnode(dev)
def exp_energy(state):
    qml.BasisState(np.array(state),wires=range(num_wires))
    return qml.expval(hamiltonian)

In [7]:
exp_energy(hf)

tensor(-1.24685995, requires_grad=True)

## 2. Prepare trial ground state
Double excitation gate -
Ansatz
![chart](./images/VQE_6.png)

In [8]:
def ansatz(params):
    qml.BasisState(hf, wires=range(num_wires))
    qml.DoubleExcitation(params[0],wires = [0,1,2,3])
    qml.DoubleExcitation(params[1], wires=[0,1,4,5])

## 3. Minimizing the expectation value of the Hamiltonian
Ritz variational principle


In [9]:
@qml.qnode(dev)
def cost_function(params):
    ansatz(params)
    return qml.expval(hamiltonian)

In [10]:
cost_function([0.1,0.1])

tensor(-1.26810417, requires_grad=True)

In [14]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)
theta = np.array([0.0,0.0], requires_grad = True)

energy = [cost_function(theta)]
angle = [theta]
max_iterations = 20

for n in range(max_iterations):
    theta,prev_energy = opt.step_and_cost(cost_function,theta)

    energy.append(cost_function(theta))
    angle.append(theta)

    if n%2 == 0:
        print(f"Step = {n}, Energy = {energy[-1]:.8f} Ha")

Step = 0, Energy = -1.26094188 Ha
Step = 2, Energy = -1.27110529 Ha
Step = 4, Energy = -1.27345238 Ha
Step = 6, Energy = -1.27399097 Ha
Step = 8, Energy = -1.27411455 Ha
Step = 10, Energy = -1.27414292 Ha
Step = 12, Energy = -1.27414944 Ha
Step = 14, Energy = -1.27415094 Ha
Step = 16, Energy = -1.27415128 Ha
Step = 18, Energy = -1.27415136 Ha


In [15]:
print("\n" f"Final ground energy: {energy[-1]: .8f}Ha")
print("\n" f"Final angle parameters: {theta[0]:.8f} {theta[1]:.8f}")


Final ground energy: -1.27415138Ha

Final angle parameters: 0.18814175 0.18891605


In [16]:
@qml.qnode(dev)
def ground_state(params):
    ansatz(params)
    return qml.state()

In [18]:
type(ground_state(theta))

pennylane.numpy.tensor.tensor